In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
import math
from scipy.stats import norm
import time
import pandas as pd
import statsmodels.api as sm
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.models import Sequential
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection
from keras.utils.np_utils import to_categorical
import os

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Train_Dataset_Landmarks

In [ ]:
address = "/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/"

In [ ]:
train_labels = os.listdir(address + "train")
test_labels = os.listdir(address + "test")

In [ ]:
print(train_labels)

['angry', 'disgust', 'fear', 'happy', 'sad', 'neutral', 'surprise']


In [ ]:
print(test_labels)

['surprise', 'neutral', 'angry', 'sad', 'disgust', 'fear', 'happy']


In [ ]:
import os
import numpy as np
import skimage.io
import skimage.draw
import dlib
from tqdm import tqdm
def extract_features(plot_landmarks = False):
    
    image_paths = []
    labels = []
    subject = os.listdir(address + "train")
    # explore the dataset and list its contents
    counter=0
    for i in range(len(train_labels)):
        imgs = os.listdir(address + 'train/'+train_labels[i])
        counter =0
        for img in imgs:
            if counter<1000 :
                image_paths.append(address + 'train/'+train_labels[i] +'/'+img)
                labels.append(i)
                counter+=1
            
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/content/drive/MyDrive/FaceEx-Recog/shape_predictor_68_face_landmarks.dat')

    landmarks = []
    if plot_landmarks:
        if not os.path.exists('landmarks'):
            os.mkdir('landmarks')
    
    landmarks.append([])    
    landmarks[-1].append([])
    indexes = []
    index=0
    for image in tqdm(image_paths):
        img = skimage.io.imread(image)
        dets = detector(img, 1)
        if (len(dets)==0):
            indexes.append(index)
            index+=1
            continue
        det = dets[0]
        shape = predictor(img, det)

        lm = np.zeros([68, 2])
        for ind in range(68):
            lm[ind] = [shape.part(ind).x, shape.part(ind).y]
            if plot_landmarks:
                rr, cc = skimage.draw.circle(lm[ind][1], lm[ind][0], 3)
                img[rr, cc] = 255
                rr, cc = skimage.draw.circle(lm[ind][1], lm[ind][0], 2)
                img[rr, cc] = 0
        landmarks[-1][-1].append(lm)
        if plot_landmarks:
            skimage.io.imsave(os.path.join('landmarks', os.path.split(image)[-1]), img)
        index+=1
    # calculate spatial features and put them in a list
    spatial_features = []
    for subject in tqdm(landmarks):
        spatial_features.append([])
        for emotion in subject:
            no_landmarks = emotion[0].shape[0]
            dist_for_two_images = []
            for image in emotion:
                dist_curr_image = np.zeros([no_landmarks * (no_landmarks - 1)])
                counter = 0
                for ind_lm1 in range(no_landmarks):
                    for ind_lm2 in range(ind_lm1 + 1, no_landmarks):
                        dist_curr_image[counter] = image[ind_lm1, 0] - image[ind_lm2, 0]
                        counter += 1
                        dist_curr_image[counter] = image[ind_lm1, 1] - image[ind_lm2, 1]
                        counter += 1
                dist_for_two_images.append(dist_curr_image)
            spatial_features[-1].append(dist_for_two_images[0] - dist_for_two_images[1])

    return landmarks, labels, spatial_features,indexes

In [ ]:
landmarks, labels, spatial_features,indexes= extract_features()

100%|██████████| 1/1 [00:13<00:00, 13.42s/it]


In [ ]:
print(landmarks[-1][-1][:10])

[array([[ 4., 16.],
       [ 4., 21.],
       [ 4., 27.],
       [ 5., 33.],
       [ 7., 39.],
       [10., 43.],
       [14., 46.],
       [19., 48.],
       [24., 48.],
       [30., 48.],
       [35., 46.],
       [40., 43.],
       [44., 39.],
       [46., 34.],
       [47., 28.],
       [48., 23.],
       [48., 17.],
       [ 8., 12.],
       [11., 10.],
       [15., 10.],
       [19., 11.],
       [23., 12.],
       [28., 12.],
       [32., 11.],
       [36., 11.],
       [40., 11.],
       [44., 13.],
       [25., 16.],
       [25., 19.],
       [25., 22.],
       [25., 25.],
       [20., 28.],
       [22., 29.],
       [25., 30.],
       [27., 29.],
       [30., 29.],
       [12., 16.],
       [15., 15.],
       [18., 15.],
       [20., 17.],
       [17., 17.],
       [14., 17.],
       [31., 17.],
       [34., 15.],
       [37., 15.],
       [39., 16.],
       [37., 17.],
       [34., 18.],
       [16., 36.],
       [19., 35.],
       [23., 35.],
       [25., 35.],
       [27.

In [ ]:
cordinate = landmarks[-1][-1][:]
x=cordinate[:]
x= np.array(x)
print(x[0])

In [ ]:
matrix = np.zeros((x.shape[0],2))

In [ ]:
print(np.mean(x[0],axis=0)[0])

25.235294117647058


In [ ]:
for i in range(x.shape[0]):
    means=np.mean(x[i],axis=0)
    matrix[i,0]=means[0]
    matrix[i,1]=means[1]

In [ ]:
print(matrix)

[[25.23529412 27.52941176]
 [24.77941176 31.48529412]
 [20.86764706 25.13235294]
 ...
 [22.89705882 27.83823529]
 [23.16176471 28.61764706]
 [23.47058824 28.38235294]]


In [ ]:
Cen_list=[]
for i in range(x.shape[0]):
    dif_list = np.zeros((x.shape[1],x.shape[2]))
    for j in range(x.shape[1]):
        dif_list[j,0]=x[i,j,0]-matrix[i,0]    
        dif_list[j,1]=x[i,j,1]-matrix[i,1]
    Cen_list.append(dif_list)

In [ ]:
print(Cen_list[:10])

In [ ]:
result_dis_angle =np.zeros((x.shape[0],x.shape[1],x.shape[2]))
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        distance = np.linalg.norm(np.asarray(x[i,j,:])-np.asarray(matrix[i,:]))
        """
        if (distance > 5 or distance <15):
          distance=0
        """
        result_dis_angle[i,j,0] = distance
        angle = (math.atan2(Cen_list[i][j,1], Cen_list[i][j,0])*360)/(2*math.pi)
        result_dis_angle[i,j,1] = angle

In [ ]:
print(x[0,0,:])

[ 4. 16.]


In [ ]:
print(matrix[0,:])

[25.23529412 27.52941176]


In [ ]:
print(result_dis_angle[0])

In [ ]:
print(np.amin(x[:,:,0]),np.amax(x[:,:,0]))
print(np.amin(x[:,:,1]),np.amax(x[:,:,1]))
print(np.amin(result_dis_angle[:,:,0]),np.amax(result_dis_angle[:,:,0]))
print(np.amin(result_dis_angle[:,:,1]),np.amax(result_dis_angle[:,:,1]))

-9.0 60.0
-9.0 59.0
0.020797258270193755 42.27753192751858
-179.96552600926663 180.0


In [ ]:
training = []
temp = []
for i in range(x.shape[0]):
    temp = []
    for j in range(x.shape[1]):
        temp.append((x[i,j,0]+9)/69)
        temp.append((x[i,j,1]+9)/69)
        temp.append((result_dis_angle[i,j,0]-0.020797258270193755)/(42.27753192751858-0.020797258270193755))
        temp.append((result_dis_angle[i,j,1]+179.96552600926663)/(180+179.96552600926663))
    training.append(temp)

In [ ]:
a1= (x[:,:,0]+9)/69
a2= (x[:,:,1]+9)/69
a3= (result_dis_angle[:,:,0]-0.020797258270193755)/(42.27753192751858-0.020797258270193755)
a4= (result_dis_angle[:,:,1]+179.96552600926663)/(180+179.96552600926663)

In [ ]:
print(np.amin(a1),np.amax(a1))
print(np.amin(a2),np.amax(a2))
print(np.amin(a3),np.amax(a3))
print(np.amin(a4),np.amax(a4))


0.0 1.0
0.0 0.9855072463768116
-0.0004921643480732221 -0.0004921643480732221
0.0 1.0


In [ ]:
for i in sorted(indexes, reverse=True): 
    del labels[i]

In [ ]:
print(len(training),len(labels))

4783 4783


# SVM

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', probability=True,verbose=True, tol=1e-3)
clf.fit(training,labels)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='poly',degree=7, probability=True,verbose=True, tol=1e-3)
clf.fit(training,labels)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=7, gamma='scale', kernel='poly',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

# Test_Dataset_Landmarks

In [ ]:
import os
import numpy as np
import skimage.io
import skimage.draw
import dlib

def extract_features(plot_landmarks = False):
    
    image_paths_t = []
    labels = []
    subject = os.listdir(address + "test")
    # explore the dataset and list its contents
    counter=0
    for i in range(len(train_labels)):
        imgs = os.listdir(address + 'test/'+train_labels[i])
        counter =0
        for img in imgs:
            if counter<200 :
                image_paths_t.append(address + 'test/'+train_labels[i] +'/'+img)
                labels.append(i)
                counter+=1
            
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/content/drive/MyDrive/FaceEx-Recog/shape_predictor_68_face_landmarks.dat')

    landmarks_t = []
    if plot_landmarks:
        if not os.path.exists('landmarks'):
            os.mkdir('landmarks')
    
    landmarks_t.append([])    
    landmarks_t[-1].append([])
    indexes_t = []
    index=0
    for image in tqdm(image_paths_t):
        img = skimage.io.imread(image)
        dets = detector(img, 1)
        if (len(dets)==0):
            indexes_t.append(index)
            index+=1
            continue
        det = dets[0]
        shape = predictor(img, det)

        lm = np.zeros([68, 2])
        for ind in range(68):
            lm[ind] = [shape.part(ind).x, shape.part(ind).y]
            if plot_landmarks:
                rr, cc = skimage.draw.circle(lm[ind][1], lm[ind][0], 3)
                img[rr, cc] = 255
                rr, cc = skimage.draw.circle(lm[ind][1], lm[ind][0], 2)
                img[rr, cc] = 0
        landmarks_t[-1][-1].append(lm)
        if plot_landmarks:
            skimage.io.imsave(os.path.join('landmarks', os.path.split(image)[-1]), img)
        index+=1
    # calculate spatial features and put them in a list
    spatial_features = []
    for subject in tqdm(landmarks_t):
        spatial_features.append([])
        for emotion in subject:
            no_landmarks = emotion[0].shape[0]
            dist_for_two_images = []
            for image in emotion:
                dist_curr_image = np.zeros([no_landmarks * (no_landmarks - 1)])
                counter = 0
                for ind_lm1 in range(no_landmarks):
                    for ind_lm2 in range(ind_lm1 + 1, no_landmarks):
                        dist_curr_image[counter] = image[ind_lm1, 0] - image[ind_lm2, 0]
                        counter += 1
                        dist_curr_image[counter] = image[ind_lm1, 1] - image[ind_lm2, 1]
                        counter += 1
                dist_for_two_images.append(dist_curr_image)
            spatial_features[-1].append(dist_for_two_images[0] - dist_for_two_images[1])

    return landmarks_t, labels, spatial_features,indexes_t

In [ ]:
landmarks_t, labels, spatial_features,indexes_t= extract_features()

100%|██████████| 1/1 [00:02<00:00,  2.38s/it]


In [ ]:
cordinate = landmarks_t[-1][-1][:]
x=cordinate[:]
x= np.array(x)
print(x.shape[0])

884


In [ ]:
matrix = np.zeros((x.shape[0],2))

In [ ]:
for i in range(x.shape[0]):
    means=np.mean(x[i],axis=0)
    matrix[i,0]=means[0]
    matrix[i,1]=means[1]

In [ ]:
Cen_list=[]
for i in range(x.shape[0]):
    dif_list = np.zeros((x.shape[1],x.shape[2]))
    for j in range(x.shape[1]):
        dif_list[j,0]=x[i,j,0]-matrix[i,0]    
        dif_list[j,1]=x[i,j,1]-matrix[i,1]
    Cen_list.append(dif_list)

In [ ]:
result_dis_angle =np.zeros((x.shape[0],x.shape[1],x.shape[2]))
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        distance = np.linalg.norm(np.asarray(x[i,j,:])-np.asarray(matrix[i,:]))
        """
        if (distance > 5 or distance <15):
          distance=0
        """
        result_dis_angle[i,j,0] = distance
        angle = (math.atan2(Cen_list[i][j,1], Cen_list[i][j,0])*360)/(2*math.pi)
        result_dis_angle[i,j,1] = angle

In [ ]:
print(np.amin(x[:,:,0]),np.amax(x[:,:,0]))
print(np.amin(x[:,:,1]),np.amax(x[:,:,1]))
print(np.amin(result_dis_angle[:,:,0]),np.amax(result_dis_angle[:,:,0]))
print(np.amin(result_dis_angle[:,:,1]),np.amax(result_dis_angle[:,:,1]))

-4.0 52.0
-4.0 58.0
0.029411764705884025 36.83160090546588
-179.9504361892549 180.0


In [ ]:
testing = []
temp = []
for i in range(x.shape[0]):
    temp = []
    for j in range(x.shape[1]):
        temp.append((x[i,j,0]+9)/69)
        temp.append((x[i,j,1]+9)/69)
        temp.append((result_dis_angle[i,j,0]-0.020797258270193755)/(42.27753192751858-0.020797258270193755))
        temp.append((result_dis_angle[i,j,1]+179.96552600926663)/(180+179.96552600926663))
    testing.append(temp)

In [ ]:
a1= (x[:,:,0]+9)/69
a2= (x[:,:,1]+9)/69
a3= (result_dis_angle[:,:,0]-0.020797258270193755)/(42.27753192751858-0.020797258270193755)
a4= (result_dis_angle[:,:,1]+179.96552600926663)/(180+179.96552600926663)

In [ ]:
print(np.amin(a1),np.amax(a1))
print(np.amin(a2),np.amax(a2))
print(np.amin(a3),np.amax(a3))
print(np.amin(a4),np.amax(a4))


0.07246376811594203 0.8840579710144928
0.07246376811594203 0.9710144927536232
0.00020386114788844127 0.8711227674197013
4.192018102126682e-05 1.0


In [ ]:
for i in sorted(indexes_t, reverse=True): 
  del labels[i]

In [ ]:
print(len(testing),len(labels))

884 884


# Accuracy on test-SVM_Landmarks

linear

In [ ]:
prediction = clf.score(testing,labels)
print(prediction)

0.3608597285067873


poly

In [ ]:
prediction = clf.score(testing,labels)
print(prediction)

0.3393665158371041


KNN-Acc_Landmarks

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(training, labels)

In [ ]:
y_pred = classifier.predict(testing)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(labels_t, y_pred))
print(classification_report(labels_t, y_pred))

[[37 42 16 17  8 11  6]
 [36 45 11  6 12 22  7]
 [21 20 18 12  8 15 19]
 [20 34  9 75 10  9  5]
 [19 41 12  7  7 18 10]
 [11 33  9  6 13 15  6]
 [19 31 17  7  9 13 30]]
              precision    recall  f1-score   support

           0       0.23      0.27      0.25       137
           1       0.18      0.32      0.23       139
           2       0.20      0.16      0.18       113
           3       0.58      0.46      0.51       162
           4       0.10      0.06      0.08       114
           5       0.15      0.16      0.15        93
           6       0.36      0.24      0.29       126

    accuracy                           0.26       884
   macro avg       0.26      0.24      0.24       884
weighted avg       0.27      0.26      0.26       884



Decision Tree-Acc_Landmarks

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(training, labels)

In [ ]:
y_pred = clf.predict(testing)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(labels_t, y_pred))
print(classification_report(labels_t, y_pred))

[[23 42 16 11 13 15 17]
 [16 59 11  9 21 13 10]
 [18 23 14  9 12 16 21]
 [14 23 18 76 13  9  9]
 [17 28 15 10 17 17 10]
 [16 31  9  3 19 10  5]
 [19 35 15  8 16 10 23]]
              precision    recall  f1-score   support

           0       0.19      0.17      0.18       137
           1       0.24      0.42      0.31       139
           2       0.14      0.12      0.13       113
           3       0.60      0.47      0.53       162
           4       0.15      0.15      0.15       114
           5       0.11      0.11      0.11        93
           6       0.24      0.18      0.21       126

    accuracy                           0.25       884
   macro avg       0.24      0.23      0.23       884
weighted avg       0.26      0.25      0.25       884



HOG

HOG-Training dataset

In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import keras
from keras_preprocessing import image
from keras import preprocessing
import matplotlib.pyplot as plt
address = '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/'

train_labels = os.listdir(address + "train")
test_labels = os.listdir(address + "test")


image_paths = []
labels = []
subject = os.listdir(address + "train")
# explore the dataset and list its contents
counter=0
for i in range(len(train_labels)):
    imgs = os.listdir(address + 'train/'+train_labels[i])
    counter =0
    for img in imgs:
        if counter<1000 :
            image_paths.append(address + 'train/'+train_labels[i] +'/'+img)
            labels.append(i)
            counter+=1

In [ ]:
print(len(image_paths))

7000


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

HOG_list = []
for i in tqdm(range(len(image_paths))):
  img = cv2.imread(image_paths[i],0)
  array = np.array([])  # empty array for storing all the features
  h = 128  # height of the image
  w = 64  # width of the image
  hog = cv2.HOGDescriptor()
  img = cv2.resize(img, (w, h), interpolation=cv2.INTER_CUBIC)  # resize images
  h = hog.compute(img, winStride=(64, 128), padding=(0, 0))  # storing HOG features as column vector
  h_trans = h.transpose()  # transposing the column vector
  array = np.vstack(h_trans)  # appending it to the array
  x = array[0,:]
  HOG_list.append(x)

100%|██████████| 7000/7000 [00:23<00:00, 293.17it/s]


In [ ]:
print(len(HOG_list))

4783


In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', probability=True,verbose=True, tol=1e-3)
clf.fit(HOG_list,labels)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

HOG-Testing dataset

In [ ]:
image_paths = []
labels = []
subject = os.listdir(address + "test")
# explore the dataset and list its contents
counter=0
for i in range(len(train_labels)):
    imgs = os.listdir(address + 'test/'+train_labels[i])
    counter =0
    for img in imgs:
        if counter<200 :
            image_paths.append(address + 'test/'+train_labels[i] +'/'+img)
            labels.append(i)
            counter+=1

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

HOG_list = []
for i in tqdm(range(len(image_paths))):
  img = cv2.imread(image_paths[i],0)
  array = np.array([])  # empty array for storing all the features
  h = 128  # height of the image
  w = 64  # width of the image
  hog = cv2.HOGDescriptor()
  img = cv2.resize(img, (w, h), interpolation=cv2.INTER_CUBIC)  # resize images
  h = hog.compute(img, winStride=(64, 128), padding=(0, 0))  # storing HOG features as column vector
  h_trans = h.transpose()  # transposing the column vector
  array = np.vstack(h_trans)  # appending it to the array
  x = array[0,:]
  x = x.tolist()
  HOG_list.append(x)

100%|██████████| 1400/1400 [00:04<00:00, 293.81it/s]


SVM-ACC-HOG

In [ ]:
prediction = clf.score(HOG_list,labels)
print(prediction)

0.24785714285714286


KNN-ACC-HOG

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(HOG_list, labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_pred = classifier.predict(HOG_list)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(labels, y_pred))
print(classification_report(labels, y_pred))

[[ 46  88  23   9   3  18  13]
 [ 36 109  16   9   2  13  15]
 [ 41  50  48   8   7  11  35]
 [ 24  58  15  74   4  12  13]
 [ 46  49  29  21  13  21  21]
 [ 31  89  17  10   9  28  16]
 [ 30  59  42   8   7  15  39]]
              precision    recall  f1-score   support

           0       0.18      0.23      0.20       200
           1       0.22      0.55      0.31       200
           2       0.25      0.24      0.25       200
           3       0.53      0.37      0.44       200
           4       0.29      0.07      0.11       200
           5       0.24      0.14      0.18       200
           6       0.26      0.20      0.22       200

    accuracy                           0.26      1400
   macro avg       0.28      0.26      0.24      1400
weighted avg       0.28      0.26      0.24      1400



Decision Tree-Acc-HOG

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(HOG_list, labels)

In [ ]:
y_pred = clf.predict(HOG_list)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(labels, y_pred))

HOG+Landmark

In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import keras
from keras_preprocessing import image
from keras import preprocessing
import matplotlib.pyplot as plt
address = '/content/drive/MyDrive/FaceEx-Recog/MMAFEDB/'

train_labels = os.listdir(address + "train")
test_labels = os.listdir(address + "test")


image_paths = []
labels = []
subject = os.listdir(address + "train")
# explore the dataset and list its contents
counter=0
for i in range(len(train_labels)):
    imgs = os.listdir(address + 'train/'+train_labels[i])
    counter =0
    for img in imgs:
        if counter<1000 :
            image_paths.append(address + 'train/'+train_labels[i] +'/'+img)
            labels.append(i)
            counter+=1

++++++++


In [ ]:
for i in sorted(indexes_t, reverse=True): 
  del HOG_list[i]

In [ ]:
for i in sorted(indexes_t, reverse=True): 
  del labels[i]

In [ ]:
print(len(labels))
print(len(HOG_list))
print(len(testing))

884
884
884


In [ ]:
hog_and_landmark = []
for i in range(len(testing)):
  hog_and_landmark.append(testing[i] + HOG_list[i])

In [ ]:
print(len(labels))
print(len(hog_and_landmark))

884
884


In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', probability=True,verbose=True, tol=1e-3)
clf.fit(hog_and_landmark,labels)

[LibSVM]

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
prediction = clf.score(hog_and_landmark,labels)
print(prediction)

0.3020361990950226
